#Boltzmann Machine

In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2023-12-17 03:02:36--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.45MB/s    in 1.4s    

2023-12-17 03:02:38 (3.45 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2023-12-17 03:02:50--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  4.11MB/s    in 1.4s    

2023-12-17 03:02:52 (4.11 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [4]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [6]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
# 사용자, 영화종류, ratings, timestamp
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Preparing the training set and the test set


In [20]:
# k-fold cross validation 위해서 u1 ~ u5의 5개 데이터셋이 존재함
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')

test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [23]:
print(f'training_set.shape: {training_set.shape}')
print(f'test_set.shape: {test_set.shape}')

training_set.shape: (79999, 4)
test_set.shape: (19999, 4)


In [22]:
training_set[:5]

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712],
       [        1,         7,         4, 875071561]])

In [24]:
test_set[:5]

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198],
       [        1,        20,         4, 887431883]])

## Getting the number of users and movies


In [25]:
# user, movie의 총 수를 구해보자
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [26]:
print(f'nb_users: {nb_users}')
print(f'nb_movies: {nb_movies}')

nb_users: 943
nb_movies: 1682


## Converting the data into an array with users in lines and movies in columns


In [27]:
def convert(data):
  new_data = []

  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]

    ratings = np.zeros(nb_movies)
    ratings[id_movies-1] = id_ratings
    new_data.append(list(ratings))

  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [31]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [32]:
# binary ratings로 바꾸기 위해서 수행
## 최종적으로 binary ratings을 예측해야 하기 때문!

training_set[training_set == 0] = -1     # 해당 영화에 사용자의 평점이 존재하지 않을 경우
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1      # 3 ~ 5점을 받았다면, '1'로 변환!

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [36]:
training_set[:10]

tensor([[-1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

In [37]:
test_set[:10]

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.]])

## Creating the architecture of the Neural Network


In [38]:
class RBM():
  # nv: number of visible node / nh: number of hidden node
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)  # randn: initialize all the weights of tensor
    self.a = torch.randn(1, nh)   # initialize bias of hidden node / shape: (batch, bias)
    self.b = torch.randn(1, nv)   # initialize bias of visible node / shape: (batch, bias)

  # hidden node
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())            # mm: 두 torch tensor를 만듦
    activation = wx + self.a.expand_as(wx)  # wx + bias / expand_as(wx): wx로 bias를 확장하는 것
    p_h_given_v = torch.sigmoid(activation) # activate the hidden node
                                                     # binary 이기 때문에 bernoulli 이용
    return p_h_given_v, torch.bernoulli(p_h_given_v) # prob 'h' given 'v'를 구한다

  # visible node
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)

    return p_v_given_h, torch.bernoulli(p_v_given_h) # 0.25 미만이라면 1(like), 0.25 이상이면 0(not like)


  # minimize energy & maximize log-likelihood gradient
  ## v0: input vector containing the rating of all the movies
  ## vk: visible node attain after k-sampling
  ## ph0: vector of probabilities
  ## phk: the probabilities of the hidden node after k-sampling given the values of the visible node 'vk'
  def train(self, v0, vk, ph0, phk):
    # torch.mm: product of two tensors
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)


nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

## Training the RBM


In [39]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.

  for id_user in range(0, nb_users - batch_size, batch_size): # stop index define
    vk = training_set[id_user : id_user + batch_size]         # id_user ~ next 100 users
    v0 = training_set[id_user : id_user + batch_size]         # 100명의 사용자에게 평가받은 평점
    ph0,_ = rbm.sample_h(v0)                                  # initial prob

    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]

    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.

  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3258)
epoch: 2 loss: tensor(0.2331)
epoch: 3 loss: tensor(0.2477)
epoch: 4 loss: tensor(0.2490)
epoch: 5 loss: tensor(0.2494)
epoch: 6 loss: tensor(0.2476)
epoch: 7 loss: tensor(0.2469)
epoch: 8 loss: tensor(0.2493)
epoch: 9 loss: tensor(0.2480)
epoch: 10 loss: tensor(0.2482)


## Testing the RBM


In [40]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]

    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.

print('test loss: '+str(test_loss/s))

test loss: tensor(0.2562)
